In [9]:
#Based on example at https://www.tensorflow.org/lite/performance/post_training_integer_quant

import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

try:
  # %tensorflow_version only exists in Colab.
  import tensorflow.compat.v2 as tf
except Exception:
  pass
tf.enable_v2_behavior()

from tensorflow import keras
import numpy as np
import pathlib
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split    
    
#A function to view an array
def view(array):
    plt.imshow(array)
    plt.xticks(ticks=[])
    plt.yticks(ticks=[])
    plt.show()

#http://localhost:8889/edit/Documents/Python/google_coral_tests/real_big_properties_df.csv 
    
prop_df = pd.read_csv('real_big_properties_df.csv').drop(columns=['Unnamed: 0'])
numbers = list(prop_df.numbers)
numbers = np.array(numbers)
shapes = list(prop_df.shapes)
shapes = np.array(shapes)
fills = list(prop_df.fills)
fills = np.array(fills)
colors = list(prop_df.colors)
colors = np.array(colors)
numbers[246] = 0
numbers[84] = 0
fills[300] = 1
numbers[300] = 1
numbers[136] = 0
numbers[298] = 0
fills[298] = 2
numbers[85] = 2
numbers[297] = 1
numbers[302] = 2
numbers[247] = 2
numbers[135] = 1
numbers[140] = 2

# Load cards and labels
#real_big_array = np.load('real_big_array.npy')
gray_big_array = np.load('gray_big_array.npy')
gray_big_array.shape

########################
#Set image size
im_size = 100

gray_small_list = []
for i in range(gray_big_array.shape[0]):
    gray_small_list.append(cv2.resize(gray_big_array[i], (im_size,im_size)))
gray_small_array = np.stack(gray_small_list, axis=0)

#######################
#######################
#Shape training
X = gray_small_array
y = fills
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.10, random_state=42)

tf.random.set_seed(42)

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(im_size, im_size)),
  keras.layers.Reshape(target_shape=(im_size, im_size, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(16),
  keras.layers.Dense(8),
  keras.layers.Dense(8),
  keras.layers.Dense(3)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=150,
  validation_split=0.1,
)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

###################
tflite_models_dir = pathlib.Path("/tmp/fill_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"fill_model.tflite"
tflite_model_file.write_bytes(tflite_model)
###################

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

images = tf.cast(gray_small_array, tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"fill_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"fill_model_quant_io.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

open("FILL_100_3.tflite", "wb").write(tflite_model)
open("FILL_post_quant_100_3.tflite", "wb").write(tflite_model_quant)

Train on 397 samples, validate on 45 samples
Epoch 1/150
397/397 [==============================] - 1s 3ms/sample - loss: 1.1679 - accuracy: 0.3652 - val_loss: 1.0925 - val_accuracy: 0.2889
Epoch 2/150
397/397 [==============================] - 1s 2ms/sample - loss: 1.0255 - accuracy: 0.4710 - val_loss: 0.9865 - val_accuracy: 0.6000
Epoch 3/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.9253 - accuracy: 0.6222 - val_loss: 0.8563 - val_accuracy: 0.6444
Epoch 4/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.7868 - accuracy: 0.6801 - val_loss: 0.7399 - val_accuracy: 0.6000
Epoch 5/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.6052 - accuracy: 0.7657 - val_loss: 0.6521 - val_accuracy: 0.6000
Epoch 6/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.4893 - accuracy: 0.7683 - val_loss: 0.4781 - val_accuracy: 0.7778
Epoch 7/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.4373 -

Epoch 57/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 1.0000
Epoch 58/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0094 - accuracy: 1.0000 - val_loss: 0.0247 - val_accuracy: 1.0000
Epoch 59/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0080 - accuracy: 0.9975 - val_loss: 0.0122 - val_accuracy: 1.0000
Epoch 60/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 61/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0087 - val_accuracy: 1.0000
Epoch 62/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.0193 - val_accuracy: 1.0000
Epoch 63/150
397/397 [==============================] - 1s 2ms/sample - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0065 -

397/397 [==============================] - 1s 2ms/sample - loss: 4.6449e-04 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 113/150
397/397 [==============================] - 1s 2ms/sample - loss: 4.2102e-04 - accuracy: 1.0000 - val_loss: 0.0050 - val_accuracy: 1.0000
Epoch 114/150
397/397 [==============================] - 1s 2ms/sample - loss: 3.8814e-04 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 115/150
397/397 [==============================] - 1s 2ms/sample - loss: 3.6555e-04 - accuracy: 1.0000 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 116/150
397/397 [==============================] - 1s 2ms/sample - loss: 3.5546e-04 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 117/150
397/397 [==============================] - 1s 2ms/sample - loss: 3.4583e-04 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 118/150
397/397 [==============================] - 1s 2ms/sample - loss: 3.4798e-04 - accuracy: 1.0000

108440